# Imports

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px 
import seaborn as sns

In [2]:
# Set display options
get_ipython().run_line_magic('matplotlib', 'inline')
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

# Data cleaning

In [ ]:
# Import the data
df = pd.read_csv(r"C:\Users\Vamsi\Downloads\NYPD_Complaint_Data_Historic_20231107.csv")

C:\Users\Vamsi\AppData\Local\Temp\ipykernel_11296\4077364714.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Vamsi\Downloads\NYPD_Complaint_Data_Historic_20231107.csv")


In [ ]:
# Display the first few rows
df.head()

In [ ]:
# Display column names
df.columns

In [ ]:
# Display summary statistics
df.describe().T

In [ ]:
# Drop useless columns
df.drop(columns=['PD_CD', 'PD_DESC', 'JURISDICTION_CODE', 'JURIS_DESC', 'KY_CD', 'RPT_DT', 'LOC_OF_OCCUR_DESC'], inplace=True)
# Drop columns with high NaN and redundant information %
df.drop(columns=['HOUSING_PSA', 'TRANSIT_DISTRICT'], inplace=True)
# Remove unnecessary coordinate columns
df.drop(columns=['X_COORD_CD', 'Y_COORD_CD', 'Lat_Lon'], inplace=True)
# Delete columns that have no event start date or time
df = df[df['CMPLNT_FR_DT'].notna() & df['CMPLNT_FR_TM'].notna()]

# Drop redundant columns
df.drop(columns=['CMPLNT_TO_TM', 'CMPLNT_TO_DT'], inplace=True)



In [ ]:
# Percentage of NaN values for each column
for column in df.columns:
    print(f'{column}: missing {df[column].isna().sum() * 100 / len(df)}')


In [ ]:
# Convert categorical columns to binary
df['IN_PARK'] = df['PARKS_NM'].notnull().astype(int)
df['IN_PUBLIC_HOUSING'] = df['HADEVELOPT'].notnull().astype(int)
df['IN_STATION'] = df['STATION_NAME'].notnull().astype(int)
df = df.rename({'PARKS_NM': 'IN_PARK', 'HADEVELOPT': 'IN_PUBLIC_HOUSING', 'STATION_NAME': 'IN_STATION'}, axis=1)


In [ ]:
# Remove unnecessary coordinate columns
df.drop(columns=['X_COORD_CD', 'Y_COORD_CD', 'Lat_Lon'], inplace=True)

# Drop rows without coordinates
df = df[df['Latitude'].notna() & df['Longitude'].notna()]


In [ ]:
# Check if the two columns have the same values
if all(df['CMPLNT_FR_DT'].notna() == df['CMPLNT_TO_DT'].notna()):
    df.drop(columns='CMPLNT_TO_DT', inplace=True)


In [ ]:
# Change date columns to YYYY-MM-DD
df['CMPLNT_FR_DT'] = pd.to_datetime(df['CMPLNT_FR_DT'], errors='coerce')
df['CMPLNT_FR_TM'] = pd.to_datetime(df['CMPLNT_FR_TM'], errors='coerce')


In [ ]:
# Split date to year, month, and day
df['year'] = df['CMPLNT_FR_DT'].dt.year
df['month'] = df['CMPLNT_FR_DT'].dt.month
df['day'] = df['CMPLNT_FR_DT'].dt.day
df['hour'] = df['CMPLNT_FR_TM'].dt.hour
# Add week day column
df['weekday'] = pd.to_datetime(df['CMPLNT_FR_DT']).dt.day_name()
df.drop(columns=['CMPLNT_FR_DT', 'CMPLNT_FR_TM'], inplace=True)


In [ ]:
# Clean suspect and victim data
df['SUSP_RACE'] = df['SUSP_RACE'].fillna('UNKNOWN')
df['VIC_RACE'] = df['VIC_RACE'].fillna('UNKNOWN')
df['SUSP_SEX'] = df['SUSP_SEX'].fillna('U')
df['VIC_SEX'] = df['VIC_SEX'].fillna('U')


In [ ]:
# Take care of invalid age data
valid_age = ['UNKNOWN', '25-44', '<18', '45-64', '65+', '18-24']
df.loc[~df['SUSP_AGE_GROUP'].isin(valid_age), 'SUSP_AGE_GROUP'] = 'UNKNOWN'
df.loc[~df['VIC_AGE_GROUP'].isin(valid_age), 'VIC_AGE_GROUP'] = 'UNKNOWN'


In [ ]:
# Drop redundant data
df.drop(columns='PATROL_BORO', inplace=True)
df['BORO_NM'] = df['BORO_NM'].fillna('UNKNOWN')


In [ ]:
# Drop remaining rows with NaN values
df.dropna(inplace=True)

In [ ]:
# Encode crime severity
df['LAW_CAT_CD'] = df['LAW_CAT_CD'].replace(['felony', 'misdemeanor', 'violation'], [2, 1, 0])
df = df.rename({'LAW_CAT_CD': 'CRIME_CLASS'}, axis=1)


In [ ]:
# Reorder columns
column_titles = ['CMPLNT_NUM', 'year', 'month', 'day', 'weekday', 'hour', 'Latitude', 'Longitude', 'CRM_ATPT_CPTD_CD',
                 'OFNS_DESC', 'ADDR_PCT_CD', 'CRIME_CLASS', 'BORO_NM', 'PREM_TYP_DESC',
                 'IN_PARK', 'IN_PUBLIC_HOUSING', 'IN_STATION',
                 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX',
                 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']
df = df.reindex(columns=column_titles)


In [ ]:
# Save the cleaned data to a CSV file
df.to_csv(r'C:\\Users\\rupesh\\Downloads\\ny_crimes_cl_data.csv', index=False)